# 1. Shift and Subtract

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import shift as ndimage_shift
import os

In [ ]:
filtername = 'F225W'
refepoch = '20200529'
shiftepoch = '20180806'
refimage = '../ASSETS/DRC_{0}/{0}_{1}_IVM_refimage20200529_drc.fits'.format(filtername,refepoch)
shiftimage = '../ASSETS/DRC_{0}/{0}_{1}_IVM_refimage20200529_drc.fits'.format(filtername,shiftepoch)
outputfolder = '../ASSETS/SHIFT/'
shiftfile = '{0}_{1}_to_{2}.txt'.format(filtername,shiftepoch,refepoch)

In [ ]:
t1 = fits.open(refimage)['SCI'].data
m = np.isfinite(t1)
vmin,vmax = np.percentile(t1[m],5.),np.percentile(t1[m],99.)
plt.figure(figsize=(10,10))
plt.imshow(t1,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax),plt.show()
#####
t2 = fits.open(shiftimage)['SCI'].data
plt.figure(figsize=(10,10))
plt.imshow(t2,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax),plt.show()
#####
t3 = t1-t2
vmin,vmax = -0.01,0.01
plt.figure(figsize=(10,10))
plt.imshow(t3,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax),plt.show()

In [ ]:
shift_xy = (3.8,-4.8)
outpath = '{0}_{1}_shifted_to_{2}_drc.fits'.format(filtername,shiftepoch,refepoch)
subpath = '{0}_{1}_sub_drz.fits'.format(filtername,shiftepoch)
replacevalue = 0.
order = 3
mode = 'nearest'
#####
image = fits.open(shiftimage)['SCI'].data.copy()
m = ~np.isfinite(image)
image[m] = replacevalue
shift_yx = (shift_xy[1],shift_xy[0])
t1 = ndimage_shift(image,shift_yx,order=order,mode=mode)
os.system('cp {0} {1}'.format(shiftimage,outpath))
t2 = fits.open(outpath)
t2['SCI'].data = t1.copy()
t2.writeto(outpath,overwrite=True)
#####
os.system('cp {0} {1}'.format(shiftimage,subpath))
t1 = fits.open(refimage)['SCI'].data.copy()
t2 = fits.open(outpath)['SCI'].data.copy()
t3 = t2 - t1
t4 = fits.open(subpath)
t4['SCI'].data = t3.copy()
t4.writeto(subpath,overwrite=True)
#####
plt.figure(figsize=(10,10))
t = fits.open(subpath)['SCI'].data.copy()
plt.imshow(t,origin='lower',cmap='viridis',vmin=-0.01,vmax=0.01)

In [ ]:
##### output info
t = outputfolder + shiftfile
f = open(t,'w')
#####
t1 = []
t1.append('{0}_{1}_to_{2}\n'.format(filtername,shiftepoch,refepoch))
t1.append('refimage {0}\n'.format(refimage))
t1.append('shiftimage {0}\n'.format(shiftimage))
t1.append('xy_shift = refimage - shiftimage\n')
t1.append('xshift {0}\n'.format(shift_xy[0]))
t1.append('yshift {0}\n'.format(shift_xy[1]))
for i in t1:
    f.writelines(i)
#####
f.close()

In [ ]:
##### Move files
os.system('mv {0} {1}'.format(outpath,outputfolder))
os.system('mv {0} {1}'.format(subpath,outputfolder))

# 2. Finding AT2018COW Centroid

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from photutils.detection import IRAFStarFinder
from photutils.psf import IntegratedGaussianPRF, DAOGroup
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_fwhm_to_sigma
from photutils.background import MMMBackground, MADStdBackgroundRMS
from photutils.psf import IterativelySubtractedPSFPhotometry

In [ ]:
filename = '../ASSETS/SHIFT/F225W_20180806_sub_drz.fits'
regfile = '../ASSETS/REGION/F225W_20180806_AT2018cow_xy.reg'
outputfile = '../ASSETS/REGION/AT2018cow_F225W_20200529_refimage_20200529_xy.dat'
filename2 = '../ASSETS/DRC_F225W/F225W_20200529_IVM_refimage20200529_drc.fits'

In [ ]:
xy_centroid = None
f = open(regfile,'r')
for ii,i in enumerate(f.readlines()):
    if ii==4:
        t = i.split('(')[1]
        t1,t2,t3 = t.split(',')
        xy_centroid = (float(t1),float(t2))
xy_centroid

In [ ]:
t = fits.open(filename)['SCI'].data
plt.figure(figsize=(10,10))
m = np.isfinite(t)
vmin,vmax = np.percentile(t[m],5.),np.percentile(t[m],99.)
plt.imshow(t,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
tx,ty = xy_centroid
plt.scatter(tx,ty,s=100,edgecolor='red',facecolor='None')
# dx,dy = 50,50
# plt.xlim(tx-dx,tx+dx)
# plt.ylim(ty-dy,ty+dy)

In [ ]:
##### recentroid
image = fits.open(filename)['SCI'].data.copy()
bkgrms = MADStdBackgroundRMS()
std = bkgrms(image)
fwhm = 2. # fwhm = 1 pixel for F105W
iraffind = IRAFStarFinder(threshold=10.*std,fwhm=fwhm,minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0,sharplo=0.0, sharphi=2.0)
daogroup = DAOGroup(2.0*fwhm)
mmm_bkg = MMMBackground()
fitter = LevMarLSQFitter()
psf_model = IntegratedGaussianPRF(sigma=fwhm*gaussian_fwhm_to_sigma)
psf_model.sigma.fixed = False
photometry = IterativelySubtractedPSFPhotometry(finder=iraffind,group_maker=daogroup,bkg_estimator=mmm_bkg,psf_model=psf_model,fitter=LevMarLSQFitter(),niters=1, fitshape=(11,11))
result_tab = photometry(image=image)
residual_image = photometry.get_residual_image()
result_tab

In [ ]:
objid = None
tx = result_tab['x_fit'] - xy_centroid[0]
ty = result_tab['y_fit'] - xy_centroid[1]
tz = np.sqrt(np.power(tx,2) + np.power(ty,2))
objid = np.argwhere(tz == tz.min()).flatten()
#####
t = fits.open(filename)['SCI'].data
plt.figure(figsize=(10,10))
m = np.isfinite(t)
vmin,vmax = np.percentile(t[m],5.),np.percentile(t[m],99.)
plt.imshow(t,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
t1 = result_tab[objid]
tx,ty = t1['x_fit'],t1['y_fit']
plt.scatter(tx,ty,s=100,edgecolor='red',facecolor='None')
dx,dy = 50,50
plt.xlim(tx-dx,tx+dx)
plt.ylim(ty-dy,ty+dy)

In [ ]:
from astropy.wcs import WCS
t = WCS(fits.open(filename)['SCI'].header)
t2 = t.pixel_to_world(tx,ty)
tra,tdec = t2.ra.degree[0],t2.dec.degree[0]

In [ ]:
f = open(outputfile,'w')
t1 = result_tab[objid]
tx,ty = t1['x_fit'],t1['y_fit']
t = []
t.append('objname AT2018cow\n')
t.append('filename {0}\n'.format(filename))
t.append('filename2 {0}\n'.format(filename2))
t.append('ra_degree {0}\n'.format(tra))
t.append('dec_degree {0}\n'.format(tdec))
t.append('x {0}\n'.format(tx[0]))
t.append('y {0}\n'.format(ty[0]))
t.append('# RADEC in ICRS, undistorted frame.\n')
t.append('# Pixel xy here is 0-index for python. (+1,+1) to DS9 image pixel. (+1,+2) to DS9 physical pixel.\n')
f.writelines(t)
f.close()

In [ ]:
t = fits.open(filename2)['SCI'].data
plt.figure(figsize=(10,10))
m = np.isfinite(t)
vmin,vmax = np.percentile(t[m],5.),np.percentile(t[m],99.)
plt.imshow(t,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
plt.scatter(tx,ty,s=100,edgecolor='red',facecolor='None')
dx,dy = 50,50
plt.xlim(tx-dx,tx+dx)
plt.ylim(ty-dy,ty+dy)

# 3. Aligning Images Using Objects in the FoVs

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from photutils.detection import IRAFStarFinder
from photutils.psf import IntegratedGaussianPRF, DAOGroup
from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_fwhm_to_sigma
from photutils.background import MMMBackground, MADStdBackgroundRMS
from photutils.psf import IterativelySubtractedPSFPhotometry
import pandas as pd

In [ ]:
filelist = ['../ASSETS/DRC_F336W/F336W_20200529_IVM_drc.fits',
            '../ASSETS/DRC_F225W/F225W_20200529_IVM_refimage20200529_drc.fits',
           ]
outfile0 = '../ASSETS/DRC_F336W/sources_F336W_20200529.csv'
outfile1 = '../ASSETS/DRC_F225W/sources_F225W_20200529.csv'
outfile = '../ASSETS/DRC_F225W/offset_F225W_20200529_to_F336W_20200529.txt'

In [ ]:
n_thres = 10.
##### recentroid
image = fits.open(filelist[0])['SCI'].data.copy()
bkgrms = MADStdBackgroundRMS()
std = bkgrms(image)
fwhm = 2. # fwhm = 1 pixel for F105W
iraffind = IRAFStarFinder(threshold=n_thres*std,fwhm=fwhm,minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0,sharplo=0.0, sharphi=2.0)
daogroup = DAOGroup(2.0*fwhm)
mmm_bkg = MMMBackground()
fitter = LevMarLSQFitter()
psf_model = IntegratedGaussianPRF(sigma=fwhm*gaussian_fwhm_to_sigma)
psf_model.sigma.fixed = False
photometry = IterativelySubtractedPSFPhotometry(finder=iraffind,group_maker=daogroup,bkg_estimator=mmm_bkg,psf_model=psf_model,fitter=LevMarLSQFitter(),niters=1, fitshape=(11,11))
result_tab = photometry(image=image)
residual_image = photometry.get_residual_image()
# result_tab
#####
tx,ty = result_tab['x_fit'],result_tab['y_fit']
m = np.isfinite(image)
vmin,vmax = np.percentile(image[m],5.),np.percentile(image[m],99.)
plt.figure(figsize=(10,10))
plt.imshow(image,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
plt.plot(tx,ty,'x',color='red')
##### to csv
t = result_tab.to_pandas()
t.to_csv(outfile0)

In [ ]:
n_thres = 10.
##### recentroid
image = fits.open(filelist[1])['SCI'].data.copy()
bkgrms = MADStdBackgroundRMS()
std = bkgrms(image)
fwhm = 2. # fwhm = 1 pixel for F105W
iraffind = IRAFStarFinder(threshold=n_thres*std,fwhm=fwhm,minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0,sharplo=0.0, sharphi=2.0)
daogroup = DAOGroup(2.0*fwhm)
mmm_bkg = MMMBackground()
fitter = LevMarLSQFitter()
psf_model = IntegratedGaussianPRF(sigma=fwhm*gaussian_fwhm_to_sigma)
psf_model.sigma.fixed = False
photometry = IterativelySubtractedPSFPhotometry(finder=iraffind,group_maker=daogroup,bkg_estimator=mmm_bkg,psf_model=psf_model,fitter=LevMarLSQFitter(),niters=1, fitshape=(11,11))
result_tab = photometry(image=image)
residual_image = photometry.get_residual_image()
# result_tab
#####
tx,ty = result_tab['x_fit'],result_tab['y_fit']
m = np.isfinite(image)
vmin,vmax = np.percentile(image[m],5.),np.percentile(image[m],99.)
plt.figure(figsize=(10,10))
plt.imshow(image,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
plt.plot(tx,ty,'x',color='red')
##### to csv
t = result_tab.to_pandas()
t.to_csv(outfile1)

In [ ]:
image = fits.open(filelist[1])['SCI'].data
plt.figure(figsize=(20,20))
plt.imshow(image,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
#####
result_tab = pd.read_csv(outfile1)
tx,ty = result_tab['x_fit'],result_tab['y_fit']
plt.plot(tx,ty,'o',color='black',alpha=0.6)
#####
result_tab = pd.read_csv(outfile0)
tx,ty = result_tab['x_fit'],result_tab['y_fit']
plt.plot(tx,ty,'x',color='red',alpha=1.,markersize=10)

In [ ]:
result_tab = pd.read_csv(outfile0)
result_tab2 = pd.read_csv(outfile1)
tx,ty = result_tab['x_fit'].values,result_tab['y_fit'].values
tx2,ty2 = result_tab2['x_fit'].values,result_tab2['y_fit'].values
match_index = []
distance = []
for ii,i in enumerate(tx):
    dx,dy = tx2-tx[ii],ty2-ty[ii]
    dd = np.sqrt(np.power(dx,2) + np.power(dy,2))
    m = np.argwhere(dd == dd.min()).flatten()
    match_index.append(m[0])
    distance.append(dd.min())

In [ ]:
##### use cutoff to filter sources
index_keep = []
im1xlist,im1ylist = [],[]
im2xlist,im2ylist = [],[]
dxlist,dylist,ddlist = [],[],[]
distance_mean = np.mean(distance)
distance_std = np.std(distance)
##### define cutoff
# n = 0.
# cutoff = distance_mean - n*distance_std
cutoff = 1.
#####
image = fits.open(filelist[1])['SCI'].data
result_tab = pd.read_csv(outfile0)
result_tab2 = pd.read_csv(outfile1)
tx,ty = result_tab['x_fit'],result_tab['y_fit']
tx2,ty2 = result_tab2['x_fit'],result_tab2['y_fit']
##### plot images with sources
plt.figure(figsize=(10,10))
plt.imshow(image,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
for ii,i in enumerate(tx):
    if distance[ii]>cutoff:
        continue
    plt.plot(tx[ii],ty[ii],'ro')
    plt.plot(tx2[match_index[ii]],ty2[match_index[ii]],'rs',lw=4)
    plt.plot([tx[ii],tx2[match_index[ii]]],[ty[ii],ty2[match_index[ii]]],'r-',lw=4)
    index_keep.append((ii,match_index[ii]))
plt.show()
##### plot distribution
i1list = []
i2list = []
for ii,i in enumerate(index_keep):
    i1,i2 = i
    dx = tx[i1] - tx2[i2]
    dy = ty[i1] - ty2[i2]
    dd = np.sqrt(np.power(dx,2)+np.power(dy,2))
    i1list.append(i1)
    i2list.append(i2)
    dxlist.append(dx)
    dylist.append(dy)
    ddlist.append(dd)
    im1xlist.append(tx[i1])
    im1ylist.append(ty[i1])
    im2xlist.append(tx2[i2])
    im2ylist.append(ty2[i2])
plt.plot(tx2[i2list],dxlist,'x'),plt.xlabel('pix x'),plt.ylabel('dx'),plt.show()
plt.plot(ty2[i2list],dxlist,'x'),plt.xlabel('pix y'),plt.ylabel('dx'),plt.show()
plt.plot(tx2[i2list],dylist,'x'),plt.xlabel('pix x'),plt.ylabel('dy'),plt.show()
plt.plot(ty2[i2list],dylist,'x'),plt.xlabel('pix y'),plt.ylabel('dy'),plt.show()
plt.plot(tx2[i2list],ddlist,'x'),plt.xlabel('pix x'),plt.ylabel('shift distance'),plt.show()
plt.plot(ty2[i2list],ddlist,'x'),plt.xlabel('pix y'),plt.ylabel('shift distance'),plt.show()
##### print summary stats
print('cutoff distance = {0}'.format(cutoff))
print('dx mean,std = {0},{1}'.format(np.mean(dxlist),np.std(dxlist)))
print('dy mean,std = {0},{1}'.format(np.mean(dylist),np.std(dylist)))
print('shift distance mean,std = {0},{1}'.format(np.mean(ddlist),np.std(ddlist)))

In [ ]:
for ii,i in enumerate(dxlist):
    print(ii,dxlist[ii],dylist[ii],ddlist[ii])

In [ ]:
##### output file
t = []
t.append('image1 {0}\n'.format(filelist[0]))
t.append('image2 {0}\n'.format(filelist[1]))
t.append('offset = image1 - image2\n')
t.append('dx_mean {0}\n'.format(np.mean(dxlist)))
t.append('dx_std {0}\n'.format(np.std(dxlist)))
t.append('dy_mean {0}\n'.format(np.mean(dylist)))
t.append('dy_std {0}\n'.format(np.std(dylist)))
t.append('shift_distance_mean {0}\n'.format(np.mean(ddlist)))
t.append('shift_distance_std {0}\n'.format(np.std(ddlist)))
t.append('# objid,im1_x,im1_y,im2_x,im2_y,dx,dy,shift_distance\n')
for ii,i in enumerate(im1xlist):
    t0 = ii
    t1,t2 = im1xlist[ii],im1ylist[ii]
    t3,t4 = im2xlist[ii],im2ylist[ii]
    t5,t6,t7 = dxlist[ii],dylist[ii],ddlist[ii]
    t.append('# {0},{1},{2},{3},{4},{5},{6},{7}\n'.format(t0,t1,t2,t3,t4,t5,t6,t7))
#####
f = open(outfile,'w')
f.writelines(t)
f.close()

# 4. Quality Check

In [ ]:
t = '../ASSETS/REGION/AT2018cow_F336W_20200529_refimage_20200529_xy.dat'
posx,posy = None,None
f = open(t,'r')
for ii,i in enumerate(f.readlines()):
    if i.split(' ')[0] == 'x':
        posx = float(i.split(' ')[1])
    if i.split(' ')[0] == 'y':
        posy = float(i.split(' ')[1])
f.close()
posx,posy

In [ ]:
posx2,posy2 = posx-np.mean(dxlist),posy-np.mean(dylist)
posx2,posy2

In [ ]:
t = '../ASSETS/REGION/AT2018cow_F225W_20200529_refimage_20200529_xy.dat'
tx,ty = None,None
f = open(t,'r')
for ii,i in enumerate(f.readlines()):
    if i.split(' ')[0] == 'x':
        tx = float(i.split(' ')[1])
    if i.split(' ')[0] == 'y':
        ty = float(i.split(' ')[1])
f.close()
tx,ty

In [ ]:
print('discrepancy between two methods: dx = {0}, dy = {1}'.format(tx-posx2,ty-posy2))